# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto - Ibrahim Volpi

#### Let's proceed to install and import all needed packages:

In [1]:
#!pip install geocoder
#!pip install lxml
#!pip install beautifulsoup4
#!pip install folium

In [2]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import numpy as np
from itertools import chain
from sklearn import metrics
import geocoder # import geocoder
from bs4 import BeautifulSoup 
import folium 
import matplotlib.pyplot as plt

#### The below code sets the location of the wikipedia web page and scrap the section where the information for the table is located

In [3]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text #request to get the html as text

In [4]:
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'}) #getting the section of the code related to the table

#### In this first part of Data Pre-processing, let's get the data from the code scrapped above cleaned by runing some "for/While" instances to get the data cleaned as much as possible from alien characters and blank spaces.

In [5]:
Vector=[]
test2=My_table.find_all("td")

n=0
while n<len(test2):
    Vector=Vector + [test2[n]]                        #getting just the part that has the table content
    n=n+1

i=0
while i<len(Vector):
    Vector[i]=str(Vector[i])                                             
    i=i+1
t=0
Vector_G=[]
while t<len(Vector):
    Vector_G=Vector_G + [[Vector[t],Vector[t+1],Vector[t+2]]]              #grouping into three coloumns
    n=n+1
    t=t+3

for i in Vector_G:
    for t in range (0,3):
        i[t]=i[t].replace("<td>",'')
        i[t]=i[t].replace("</td>",'')                                     #eliminating extra char. not needed
        i[t]=i[t].replace("\n",'')
        i[t]=i[t].replace('<a href="/wiki/','')      
        i[t]=i[t].replace('</a>','')
        i[t]=i[t].replace('>',' ')

x=0
while x<len(Vector_G):
    Vector_G[x][1]=Vector_G[x][1].split('"',-1)[-1]
    Vector_G[x][2]=Vector_G[x][2].split('"',-1)[-1]                         #recovering the Borough and Neighborhoods        
    x=x+1

x=0
i=0
while x<len(Vector_G):
    while i<3:
        Vector_G[x][i]=Vector_G[x][i].strip()                         #deleting blank spaces before and after each value
        i=i+1
    i=0
    x=x+1   

#### Once the data is being prepared, lets create a Pandas DataFrame and make sure to fulfill the requirements as per the assignment: 

-  a) rows with Borough as "Not Assigned" will be dropped.
- b) rows with Neigbourhood as "Not assigned" will be assigned with the Borough name.

In [6]:
Df=pd.DataFrame(Vector_G)
Df.columns = ['PostalCode', 'Borough', 'Neighborhood']              #setting the headers
Df2=Df.copy()
Df = Df[Df.Borough != 'Not assigned']                               #dropping the rows with Borough='Not assigned'
lt=Df.values.tolist()                                 #converting to list to add the borough as neigh. when not assigned
for i in lt:
    if i[2]=='Not assigned':
        i[2]=i[1]
Df=pd.DataFrame(lt)                                            #converting to DataFrame again, setting headers again
Df.columns = ['PostalCode', 'Borough', 'Neighborhood']           

#### Data sharing the same Postal Code will be merged in one row.    

In [7]:
aggregation_functions = {'PostalCode': 'first', 'Borough': 'first', 'Neighborhood': ", ".join}
Df_group = Df.groupby(Df['PostalCode']).aggregate(aggregation_functions)
Df=Df_group.reset_index(drop='true')               #combining the coloumns with same post code, reseting index afterwards

In [8]:
Df[0:10]                                        #showing the first ten rows

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [9]:
Df.shape                       #informing the shape as per requested.

(103, 3)

#### Up to this point would be the first part of the assignment
#### Let's now recover the Latitud and Longitud for each Postal Code using the Geocoder Package

In [10]:
lat_lng_coords = []
code=Df["PostalCode"].tolist()                                           #Creates a list with the coord. and the Codes

for i in code:
    g = geocoder.arcgis('{0}, Toronto, Ontario'.format(i))
    lat_lng_coords = lat_lng_coords + [[i] + g.latlng]
    
  

#### DataFrame created using the coordinates 

In [11]:
Geocode=pd.DataFrame(lat_lng_coords)    #Creates a DataFrame from the previous list
Geocode.columns=['PostalCode', 'Latitude','Longitude']  #Define columns

In [12]:
Geocode.head()

,PostalCode,Latitude,Longitude
0,M1B,43.811525,-79.195517
1,M1C,43.785665,-79.158725
2,M1E,43.765815,-79.175193
3,M1G,43.768369,-79.217590
4,M1H,43.769688,-79.239440


#### New DataFrame thats concatenate the new information with our previous DataFrame

In [1]:
Df=pd.concat([Df, Geocode['Latitude'], Geocode['Longitude']], axis=1) #concatenate the tables
Df.head()

NameError: name 'pd' is not defined